In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/sample.csv.csv
/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv
/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv


In [2]:
rdf = pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv")
rdf.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-06-28 17:20:21,2023-06-28 16:34:45,1.0,2.14,1.0,N,120,9,Credit Card,2.5,7.165589,0.0,1.0,20.64,2.5,0.0
1,0,2023-06-29 23:05:01,2023-06-29 22:01:35,1.0,2.70,1.0,N,15,215,Credit Card,3.5,6.067401,0.0,1.0,25.55,2.5,0.0
2,1,2023-06-30 10:19:31,2023-06-30 11:13:10,1.0,1.15,1.0,N,167,223,Credit Card,0.0,4.111547,0.0,1.0,17.64,2.5,0.0
3,0,2023-06-29 13:23:09,2023-06-29 14:20:01,1.0,0.40,1.0,N,128,239,Credit Card,2.5,6.411079,0.0,1.0,12.80,2.5,0.0
4,1,2023-06-29 22:03:32,2023-06-29 22:22:22,3.0,1.10,1.0,N,203,52,Credit Card,1.0,4.769377,0.0,1.0,18.00,2.5,0.0
5,0,2023-06-29 16:31:14,2023-06-29 16:29:06,1.0,0.00,99.0,N,225,256,Credit Card,0.0,1.507831,0.0,1.0,44.70,0.0,0.0
6,1,2023-06-30 12:54:57,2023-06-30 11:52:20,1.0,1.33,1.0,N,214,240,Cash,0.0,0.094334,0.0,1.0,18.20,2.5,0.0
7,1,2023-06-30 17:16:44,2023-06-30 17:40:34,1.0,0.45,1.0,N,176,227,Cash,2.5,0.727105,0.0,1.0,13.70,2.5,0.0
8,0,2023-06-30 21:52:44,2023-06-30 21:54:13,3.0,3.80,1.0,N,196,139,Credit Card,3.5,6.590745,0.0,1.0,35.60,2.5,0.0
9,1,2023-06-29 11:12:12,2023-06-29 09:39:24,1.0,2.50,1.0,N,138,196,Credit Card,0.0,6.091603,0.0,1.0,23.10,2.5,0.0


In [3]:
#Converting timestamps to datetime object
rdf['tpep_pickup_datetime'] = pd.to_datetime(rdf['tpep_pickup_datetime'])
rdf['tpep_dropoff_datetime'] = pd.to_datetime(rdf['tpep_dropoff_datetime'])

#calculating the duration of each ride
rdf['duration'] = (rdf['tpep_dropoff_datetime'] - rdf['tpep_pickup_datetime']).dt.total_seconds()
rdf.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-06-28 17:20:21,2023-06-28 16:34:45,1.0,2.14,1.0,N,120,9,Credit Card,2.5,7.165589,0.0,1.0,20.64,2.5,0.0,-2736.0
1,0,2023-06-29 23:05:01,2023-06-29 22:01:35,1.0,2.70,1.0,N,15,215,Credit Card,3.5,6.067401,0.0,1.0,25.55,2.5,0.0,-3806.0
2,1,2023-06-30 10:19:31,2023-06-30 11:13:10,1.0,1.15,1.0,N,167,223,Credit Card,0.0,4.111547,0.0,1.0,17.64,2.5,0.0,3219.0
3,0,2023-06-29 13:23:09,2023-06-29 14:20:01,1.0,0.40,1.0,N,128,239,Credit Card,2.5,6.411079,0.0,1.0,12.80,2.5,0.0,3412.0
4,1,2023-06-29 22:03:32,2023-06-29 22:22:22,3.0,1.10,1.0,N,203,52,Credit Card,1.0,4.769377,0.0,1.0,18.00,2.5,0.0,1130.0


In [4]:
#dropping the datetime columns after calculating duration and imputing it with mean

rdf = rdf.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'store_and_fwd_flag'])

rdf.head()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,1.0,2.14,1.0,120,9,Credit Card,2.5,7.165589,0.0,1.0,20.64,2.5,0.0,-2736.0
1,0,1.0,2.70,1.0,15,215,Credit Card,3.5,6.067401,0.0,1.0,25.55,2.5,0.0,-3806.0
2,1,1.0,1.15,1.0,167,223,Credit Card,0.0,4.111547,0.0,1.0,17.64,2.5,0.0,3219.0
3,0,1.0,0.40,1.0,128,239,Credit Card,2.5,6.411079,0.0,1.0,12.80,2.5,0.0,3412.0
4,1,3.0,1.10,1.0,203,52,Credit Card,1.0,4.769377,0.0,1.0,18.00,2.5,0.0,1130.0


In [5]:
#Calculating the number of negative durations observed
rdf['duration'].loc[rdf['duration']>=0].count()

109326

In [6]:
# Calculate the mean duration excluding the negative values
median_duration = rdf[rdf['duration'] >= 0]['duration'].median() 
print(median_duration)
# Replace the negative values with the calculated median
rdf.loc[rdf['duration'] < 0, 'duration'] = median_duration

2574.0


In [7]:
rdf.head() #574.0 is the median of positive values

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,1.0,2.14,1.0,120,9,Credit Card,2.5,7.165589,0.0,1.0,20.64,2.5,0.0,2574.0
1,0,1.0,2.70,1.0,15,215,Credit Card,3.5,6.067401,0.0,1.0,25.55,2.5,0.0,2574.0
2,1,1.0,1.15,1.0,167,223,Credit Card,0.0,4.111547,0.0,1.0,17.64,2.5,0.0,3219.0
3,0,1.0,0.40,1.0,128,239,Credit Card,2.5,6.411079,0.0,1.0,12.80,2.5,0.0,3412.0
4,1,3.0,1.10,1.0,203,52,Credit Card,1.0,4.769377,0.0,1.0,18.00,2.5,0.0,1130.0


In [8]:
rdf.isnull().sum() #checking the number of negative values in other fields

VendorID                    0
passenger_count          6077
trip_distance               0
RatecodeID               6077
PULocationID                0
DOLocationID                0
payment_type                0
extra                       0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
total_amount                0
congestion_surcharge     6077
Airport_fee              6077
duration                    0
dtype: int64

In [9]:
for col in rdf:
    print(col,':',len(rdf[col].unique())) #checking the no. of unique values of each field

VendorID : 3
passenger_count : 10
trip_distance : 2929
RatecodeID : 7
PULocationID : 264
DOLocationID : 264
payment_type : 5
extra : 28
tip_amount : 175000
tolls_amount : 214
improvement_surcharge : 4
total_amount : 8164
congestion_surcharge : 4
Airport_fee : 4
duration : 8960


In [10]:
df_encoded = pd.get_dummies(rdf, columns=['payment_type'], dtype='int')

# Output the DataFrame with one-hot encoding using 0 and 1
df_encoded.head()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,extra,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration,payment_type_Cash,payment_type_Credit Card,payment_type_UPI,payment_type_Wallet,payment_type_unknown
0,1,1.0,2.14,1.0,120,9,2.5,7.165589,0.0,1.0,20.64,2.5,0.0,2574.0,0,1,0,0,0
1,0,1.0,2.70,1.0,15,215,3.5,6.067401,0.0,1.0,25.55,2.5,0.0,2574.0,0,1,0,0,0
2,1,1.0,1.15,1.0,167,223,0.0,4.111547,0.0,1.0,17.64,2.5,0.0,3219.0,0,1,0,0,0
3,0,1.0,0.40,1.0,128,239,2.5,6.411079,0.0,1.0,12.80,2.5,0.0,3412.0,0,1,0,0,0
4,1,3.0,1.10,1.0,203,52,1.0,4.769377,0.0,1.0,18.00,2.5,0.0,1130.0,0,1,0,0,0


In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numerical_cols = ['VendorID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'RatecodeID', 'extra', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'Airport_fee', 'total_amount', 'duration']

# Create transformers for numerical columns
numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])


# Create a ColumnTransformer to apply transformers to the appropriate columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols)
    ], remainder='passthrough')

# Apply the preprocessing to the DataFrame
X = preprocessor.fit_transform(df_encoded)

df = pd.DataFrame(X, columns=numerical_cols + [col for col in df_encoded.columns if col not in numerical_cols])

df.head()

,VendorID,PULocationID,DOLocationID,passenger_count,trip_distance,RatecodeID,extra,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,total_amount,duration,payment_type_Cash,payment_type_Credit Card,payment_type_UPI,payment_type_Wallet,payment_type_unknown
0,1.0,120.0,9.0,1.0,2.14,1.0,2.5,7.165589,0.0,1.0,2.5,0.0,20.64,2574.0,0.0,1.0,0.0,0.0,0.0
1,0.0,15.0,215.0,1.0,2.70,1.0,3.5,6.067401,0.0,1.0,2.5,0.0,25.55,2574.0,0.0,1.0,0.0,0.0,0.0
2,1.0,167.0,223.0,1.0,1.15,1.0,0.0,4.111547,0.0,1.0,2.5,0.0,17.64,3219.0,0.0,1.0,0.0,0.0,0.0
3,0.0,128.0,239.0,1.0,0.40,1.0,2.5,6.411079,0.0,1.0,2.5,0.0,12.80,3412.0,0.0,1.0,0.0,0.0,0.0
4,1.0,203.0,52.0,3.0,1.10,1.0,1.0,4.769377,0.0,1.0,2.5,0.0,18.00,1130.0,0.0,1.0,0.0,0.0,0.0


In [12]:
df.isnull().sum()

VendorID                    0
PULocationID                0
DOLocationID                0
passenger_count             0
trip_distance               0
RatecodeID                  0
extra                       0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
congestion_surcharge        0
Airport_fee                 0
total_amount                0
duration                    0
payment_type_Cash           0
payment_type_Credit Card    0
payment_type_UPI            0
payment_type_Wallet         0
payment_type_unknown        0
dtype: int64

In [13]:
# Splitting the dataset 
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val = train_test_split(df.drop(['total_amount'],axis =1),df['total_amount'],test_size = 0.25,random_state = 42)

In [14]:
print(x_train.shape,y_train.shape)
print(x_val.shape,y_val.shape)

(131250, 18) (131250,)
(43750, 18) (43750,)


In [15]:
# from sklearn.dummy import DummyRegressor
# regressor = DummyRegressor(strategy='median')

# regressor.fit(x_train,y_train)
#_______________________________________________________________________________________________________

#Linear Regression Model
# from sklearn.linear_model import LinearRegression
# model = LinearRegression()
# model.fit(x_train, y_train)
#_______________________________________________________________________________________________________

# #Linear Regression with cross validation
# from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import LinearRegression

# model = LinearRegression()

# # Perform 5-fold cross-validation
# scores = cross_val_score(model, x_train, y_train, cv=3, scoring='r2')

# # Print R-squared scores for each fold
# print("R-squared scores for each fold:", scores)

# # Calculate the mean R-squared score
# mean_r2 = np.mean(scores)
# print("Mean R-squared score:", mean_r2)

#Linear Regression with CV score
#R-squared scores for each fold: [0.72435907 0.71817095 0.71127509]
#Mean R-squared score: 0.7179350339576548


# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LinearRegression

# model = LinearRegression()
#_______________________________________________________________________________________________________

# # Define the hyperparameter grid to search
# param_grid = {
#     'fit_intercept': [True, False],  # Include or exclude the intercept term
# }

# # For RandomizedSearchCV:
# from sklearn.model_selection import RandomizedSearchCV
# grid_search = GridSearchCV(model, param_grid, cv=3, scoring='r2')
# # Fit the model with cross-validation
# grid_search.fit(x_train, y_train)

# # Get the best hyperparameters
# best_params = grid_search.best_params_
# print("Best Hyperparameters:", best_params)

# # Get the best estimator (model)
# best_model = grid_search.best_estimator_
#_______________________________________________________________________________________________________


In [16]:
#Random Forest Regression Model
from sklearn.ensemble import RandomForestRegressor

# model = RandomForestRegressor(n_estimators=100) 0.9497
model = RandomForestRegressor(n_estimators=150, random_state=0) #0.9518
# model = RandomForestRegressor(n_estimators=200) 0.9505
model.fit(x_train, y_train)

#_______________________________________________________________________________________________________

# # Define the hyperparameter grid to search
# param_grid = {
#     'n_estimators': [10,50,100],
#     'criterion': ['squared_error', 'absolute_error'],
#     'max_features': ['sqrt', 'log2'],
#     'bootstrap':[True, False]
# }

# # For RandomizedSearchCV:
# from sklearn.model_selection import RandomizedSearchCV
# grid_search = RandomizedSearchCV(model, param_grid, cv=3, scoring='r2')
# # Fit the model with cross-validation
# grid_search.fit(x_train, y_train)

# # Get the best estimator (model)
# best_model = grid_search.best_estimator_

RandomForestRegressor(n_estimators=150, random_state=0)

In [17]:
# #KNN Regressor
# from sklearn.neighbors import KNeighborsRegressor

# model = KNeighborsRegressor()
# model.fit(x_train, y_train) #0.35911

In [18]:
y_pred = model.predict(x_val)

In [19]:
from sklearn.metrics import r2_score, mean_squared_error
# print('R2 Score', r2_score(y_val, y_pred))
# print('Mean Squared Error', mean_squared_error(y_val, y_pred))

#Linear Regression Score
#R2 Score 0.7233705329547794
#Mean Squared Error 0.278478439599588

# Calculate R-squared score with the best model
print('R2 Score', r2_score(y_val, y_pred))

R2 Score 0.9500769687852874


In [20]:
rx_test=pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv")

#Converting timestamps to datetime object
rx_test['tpep_pickup_datetime'] = pd.to_datetime(rx_test['tpep_pickup_datetime'])
rx_test['tpep_dropoff_datetime'] = pd.to_datetime(rx_test['tpep_dropoff_datetime'])

#calculating the duration of each ride
rx_test['duration'] = (rx_test['tpep_dropoff_datetime'] - rx_test['tpep_pickup_datetime']).dt.total_seconds()


#dropping the datetime columns after calculating duration and imputing it with mean
#dropping store_and_fwd_flag also
rx_test = rx_test.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'store_and_fwd_flag'])

# Calculate the mean duration excluding the negative values
median_duration = rx_test[rx_test['duration'] >= 0]['duration'].median() 

# Replace the negative values with the calculated median
rx_test.loc[rx_test['duration'] < 0, 'duration'] = median_duration

In [21]:
df_encoded_test = pd.get_dummies(rx_test, columns=['payment_type'], dtype='int')

In [22]:
#_______________________________________________________________________________________________________
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numerical_cols_test = ['VendorID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'RatecodeID', 'extra', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'Airport_fee', 'duration']

# Create transformers for numerical columns
numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])


# Create a ColumnTransformer to apply transformers to the appropriate columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols_test)
    ], remainder='passthrough')

# Apply the preprocessing to the DataFrame
X_test = preprocessor.fit_transform(df_encoded_test)

x_test = pd.DataFrame(X_test, columns = numerical_cols_test + [col for col in df_encoded_test.columns if col not in numerical_cols_test])

x_test.head()

,VendorID,PULocationID,DOLocationID,passenger_count,trip_distance,RatecodeID,extra,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,duration,payment_type_Cash,payment_type_Credit Card,payment_type_UPI,payment_type_Wallet,payment_type_unknown
0,1.0,20.0,3.0,1.0,4.95,1.0,1.0,6.067612,0.0,1.0,2.5,0.0,240.0,0.0,1.0,0.0,0.0,0.0
1,1.0,9.0,81.0,1.0,2.10,1.0,2.5,6.191269,0.0,1.0,2.5,0.0,510.0,0.0,1.0,0.0,0.0,0.0
2,1.0,92.0,90.0,1.0,0.95,1.0,2.5,3.983872,0.0,1.0,2.5,0.0,4004.0,1.0,0.0,0.0,0.0,0.0
3,0.0,19.0,102.0,1.0,0.80,1.0,3.5,6.839341,0.0,1.0,2.5,0.0,747.0,0.0,1.0,0.0,0.0,0.0
4,1.0,131.0,229.0,1.0,4.01,1.0,0.0,1.468943,0.0,1.0,0.0,0.0,1703.0,1.0,0.0,0.0,0.0,0.0


In [23]:
y_pred_test = model.predict(x_test)

In [24]:
sub = pd.DataFrame(columns = ["ID","total_amount"])
sub["ID"] = [i for i in range(1,len(y_pred_test)+1)]
sub["total_amount"] = y_pred_test
sub.to_csv('submission.csv',index=False)